In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt
from scipy import stats,optimize
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import r2_score,mean_squared_error
import math


In [ ]:
#right now this gets data locally - will add API support soon

In [4]:
weather_spiro = pd.read_csv('weather_spiro.csv',parse_dates=['time'])
weather_spiro.head()

,barometricPressure,precipitationLast3Hours,relativeHumidity,temperature,time,Time,PEF(l/min),FEV1(l),unixDate,unixTime
0,103010.0,0.0,41.008829,0.0,2020-02-21 00:51:00,1900-01-01 00:08:00,415,2.89,18313.0,480.0
1,103150.0,0.0,40.326484,-2.2,2020-02-21 03:51:00,1900-01-01 03:31:00,388,2.54,18313.0,12660.0
2,102200.0,0.0,18.705038,10.0,2020-02-22 19:51:00,1900-01-01 19:32:00,476,2.78,18314.0,70320.0
3,102300.0,0.0,66.125542,0.6,2020-02-23 05:51:00,1900-01-01 05:13:00,294,1.62,18315.0,18780.0
4,102340.0,0.0,35.825672,9.4,2020-02-23 15:51:00,1900-01-01 15:37:00,384,2.49,18315.0,56220.0


In [5]:
#find what sorts of reports are available for what area
#https://docs.airnowapi.org/Data/query
#http://www.airnowapi.org/aq/data/?startDate=2020-02-21T13&endDate=2020-03-26T14&parameters=OZONE,PM25,PM10,CO,NO2,SO2&BBOX=-73.870000,40.696253,-73.792409,40.762596&dataType=B&format=text/csv&verbose=1&nowcastonly=0&includerawconcentrations=1&API_KEY=
#to-do learn api format https://docs.airnowapi.org/Data/docs
#get other pollution concentrations
colnames=["Latitude","Longitude","UTC","Parameter","Concentration","Unit","Raw Concentration","AQI","Category",
    "Site Name","Site Agency","AQS ID","Full AQS ID"] 
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%dT%H:%M')

#df = pd.read_csv(infile, parse_dates=['datetime'], date_parser=dateparse)

AQI = pd.read_csv('./Output.csv',parse_dates=['UTC'], date_parser=dateparse,names=colnames, header=None)
AQI = AQI[["UTC","Parameter","Concentration","AQI","Site Name"]]
#we have hourly results but we won't use all that data yet
AQI['UTC']=AQI['UTC'].apply(lambda x: datetime(year=x.year, month=x.month, day=x.day))
AQI['unixDate']= AQI['UTC'].astype(np.int64)// 10**9
AQI['unixDate'] = AQI['unixDate']/86400

AQITotal = AQI[['unixDate','AQI']]
AQITotal.index = AQITotal['unixDate']
AQITotal = AQITotal.groupby(AQITotal['unixDate']).mean()
#AQITotal = AQITotal.drop(['unixDate'], axis=1)
AQI = AQI.drop(['AQI'], axis=1)


In [7]:
def formatPollutionTables(pollutant,site=""):
    table = AQI.copy()
    table = table.where(table['Parameter']==pollutant)
    table = table.rename(columns={"Concentration":pollutant+site})
    if (pollutant == 'PM2.5'):
        table = table.where(table["Site Name"]==site).dropna()
    return table.groupby(table['unixDate']).mean()

ozone = formatPollutionTables("OZONE")
PM25 = formatPollutionTables("PM2.5","Queens")
PM25Road = formatPollutionTables("PM2.5","Queens Near-road")

In [9]:
PM25.head()

,PM2.5Queens
unixDate,
18313.0,3.372727
18314.0,7.404167
18315.0,10.737500
18316.0,13.283333
18317.0,9.429167


In [12]:
weather_spiro.index=weather_spiro['unixDate']

all_data = ozone.join(PM25).join(PM25Road).join(AQITotal).join(weather_spiro).dropna()
all_data = all_data.drop(['Time'],axis=1)
all_data.head()

,OZONE,PM2.5Queens,PM2.5Queens Near-road,AQI,barometricPressure,precipitationLast3Hours,relativeHumidity,temperature,time,PEF(l/min),FEV1(l),unixDate,unixTime
unixDate,,,,,,,,,,,,,
18313.0,30.818182,3.372727,5.772727,22.272727,103010.0,0.0,41.008829,0.0,2020-02-21 00:51:00,415,2.89,18313.0,480.0
18313.0,30.818182,3.372727,5.772727,22.272727,103150.0,0.0,40.326484,-2.2,2020-02-21 03:51:00,388,2.54,18313.0,12660.0
18314.0,22.291667,7.404167,7.479167,27.638889,102200.0,0.0,18.705038,10.0,2020-02-22 19:51:00,476,2.78,18314.0,70320.0
18315.0,19.000000,10.737500,10.116667,34.902778,102300.0,0.0,66.125542,0.6,2020-02-23 05:51:00,294,1.62,18315.0,18780.0
18315.0,19.000000,10.737500,10.116667,34.902778,102340.0,0.0,35.825672,9.4,2020-02-23 15:51:00,384,2.49,18315.0,56220.0


In [15]:
X

,const,OZONE,PM2.5Queens,PM2.5Queens Near-road,AQI,barometricPressure,precipitationLast3Hours,relativeHumidity,temperature,time,unixDate,unixTime
unixDate,,,,,,,,,,,,
18313.0,1.0,30.818182,3.372727,5.772727,22.272727,103010.0,0.0000,41.008829,0.0,2020-02-21 00:51:00,18313.0,480.0
18313.0,1.0,30.818182,3.372727,5.772727,22.272727,103150.0,0.0000,40.326484,-2.2,2020-02-21 03:51:00,18313.0,12660.0
18314.0,1.0,22.291667,7.404167,7.479167,27.638889,102200.0,0.0000,18.705038,10.0,2020-02-22 19:51:00,18314.0,70320.0
18315.0,1.0,19.000000,10.737500,10.116667,34.902778,102300.0,0.0000,66.125542,0.6,2020-02-23 05:51:00,18315.0,18780.0
18315.0,1.0,19.000000,10.737500,10.116667,34.902778,102340.0,0.0000,35.825672,9.4,2020-02-23 15:51:00,18315.0,56220.0
18316.0,1.0,15.666667,13.283333,13.137500,40.250000,101970.0,0.0000,64.445125,3.9,2020-02-24 04:51:00,18316.0,15240.0
18316.0,1.0,15.666667,13.283333,13.137500,40.250000,101830.0,0.0000,60.763303,9.4,2020-02-24 15:51:00,18316.0,55560.0
18317.0,1.0,14.375000,9.429167,10.154167,31.708333,101360.0,0.0000,96.601587,6.1,2020-02-25 06:51:00,18317.0,23160.0
18317.0,1.0,14.375000,9.429167,10.154167,31.708333,101290.0,0.0000,92.782036,8.3,2020-02-25 15:51:00,18317.0,54720.0


In [16]:
Y = all_data['PEF(l/min)']
X = all_data[['OZONE','PM2.5Queens']]
X = sm.add_constant(X)

mod = sm.OLS(Y,X)
fii = mod.fit()
p_values = fii.summary()
p_values

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PEF(l/min)   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.2191
Date:                Fri, 03 Apr 2020   Prob (F-statistic):              0.803
Time:                        13:24:48   Log-Likelihood:                -1128.0
No. Observations:                 195   AIC:                             2262.
Df Residuals:                     192   BIC:                             2272.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         356.8015     61.462      5.805      0.000     235.575     478.028
OZONE           0.8726      1.606      0.543      0.588      -2.295       4.040
PM2.5Queens     0.7555      3.595      0.210      0.834      -6.335       7.846
==============================================================================
Omnibus:                        3.939   Durbin-Watson:                   1.660
Prob(Omnibus):                  0.140   Jarque-Bera (JB):                2.494
Skew:                           0.025   Prob(JB):                        0.287
Kurtosis:                       2.448   Cond. No.                         329.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""